In [1]:
#import cx_Oracle
import pandas as pd
import numpy as np
import glob
import time
from datetime import datetime, timedelta
import os
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import seaborn as sns
from matplotlib.colors import ListedColormap
from datetime import datetime
import mysql.connector
from mysql.connector import Error
from sqlalchemy import create_engine
import mysql.connector as sql
import mysql.connector as connection
import warnings
warnings.filterwarnings('ignore')  # Desactivar los warnings
inicio = time.time()

### CARGAMOS MUESTRAS PARA GENERAR LAS GRAFICAS DE ESPECTROGRAMA

In [2]:
# Conectar a la base de datos MySQL
connection = mysql.connector.connect(
    host='localhost',   # Cambia si es necesario
    user='root',  # Reemplaza con tu usuario de MySQL
    password='AAAAAAAAA',  # Reemplaza con tu contraseña de MySQL
    database='kpis_4g'  # Nombre de tu base de datos
)

# Consulta para obtener los datos de la tabla interf_lte_data
query = "SELECT * FROM interf_lte_data"

# Leer los datos en un DataFrame
df_prb = pd.read_sql(query, connection)

# Cerrar la conexión
connection.close()

In [3]:
df_prb.drop_duplicates(inplace=True)

In [1]:
#df_prb

In [5]:
# Crear una nueva columna concatenando DISTINTIVO, FECHA y HORA
df_prb['DISTINTIVO_FECHA_HORA'] = df_prb['DISTINTIVO'].astype(str) + '_' + df_prb['FECHA'].astype(str) + '_' + df_prb['HORA'].astype(str)

In [2]:
#df_prb

In [ ]:
import mysql.connector
import pandas as pd

try:
    # Establecer conexión con la base de datos
    mydb = mysql.connector.connect(
        host="localhost", 
        database='kpis_4g', 
        user="root", 
        password="AAAAAAAA"  # Cambié 'passwd' por 'password'
    )

    # Consulta SQL para seleccionar datos de la tabla freq_4g
    query = "SELECT * FROM freq_4g"

    # Leer los datos en un DataFrame
    df_freq_4g = pd.read_sql(query, mydb)

    # Cerrar la conexión después de cargar los datos
    mydb.close()

except Exception as e:
    # Si ocurrió un error, cerrar la conexión solo si fue establecida
    if mydb.is_connected():
        mydb.close()
    # Imprimir el error
    print(str(e))

In [ ]:
df_freq_4g['Earfcn_DL'] = df_freq_4g['Earfcn_DL'].astype(int)
df_freq_4g['frec_ini_ul'] = df_freq_4g['frec_ini_ul'].astype(float)

In [ ]:
df_freq_4g = df_freq_4g.rename(columns={'Earfcn_DL': 'DLEARFCN'})
df_freq_4g['ini_ul'] = df_freq_4g['frec_ini_ul'] + df_freq_4g['guarda_izq']

In [ ]:
df_interf_prb = df_prb

In [ ]:
df_interf_prb.rename(columns={'CELLNAME': 'CELLNAME_X', 'DISTINTIVO_FECHA_HORA': 'CELLNAME'}, inplace=True)

In [ ]:
df_interf_prb = df_interf_prb[['SITE_NAME', 'SITE_ADDRESS', 'CELLNAME', 'DISTINTIVO', 'CARRIER',
       'RESULT_TIME', 'FECHA', 'HORA', 'NIVEL_PRB',
       'L_UL_INTERFERENCE_AVG_PRB', 'DLEARFCN']]

In [ ]:
import pandas as pd
from sqlalchemy import create_engine

# Configurar la conexión a MySQL (reemplaza con tus credenciales)
db_user = 'root'
db_password = 'AAAAAAAAA'
db_host = 'localhost'  # o la IP de tu servidor MySQL
db_name = 'kpis_4g'
table_name = 'interf_lte_etl'

# Crear el motor de conexión usando SQLAlchemy
engine = create_engine(f'mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}')

# Definir tamaño de trozo (cantidad de filas por inserción)
chunk_size = 10000  # Ajusta según tu necesidad

# Obtener la longitud del DataFrame
n = len(df_interf_prb)

# Iterar por bloques y subir cada bloque de datos
for i in range(0, n, chunk_size):
    # Obtener el DataFrame del trozo actual
    df_chunk = df_interf_prb.iloc[i:i+chunk_size]
    
    try:
        # Subir el bloque actual a la base de datos
        df_chunk.to_sql(name=table_name, con=engine, if_exists='append', index=False)
        #print(f"Subido el bloque {i // chunk_size + 1} de {(n // chunk_size) + 1}")
    except Exception as e:
        print(f"Error al subir el bloque {i // chunk_size + 1}: {e}")
